In [1]:
import numpy as np
import pandas as pd
import sklearn.neighbors

In [2]:
data_airbnb = pd.read_csv('Cleaned_Airbnb.csv')
data_airbnb_copy = data_airbnb.copy()
data_airbnb_copy = data_airbnb_copy.drop(['Unnamed: 0'], axis = 1)
data_airbnb.head()

,Unnamed: 0,id,name,latitude,longitude,room_type,price,availability_365
0,0,2595,Skylit Midtown Castle,40.75362,-73.98377,Entire home/apt,175,365
1,1,3831,"Whole flr w/private bdrm, bath & kitchen(pls r...",40.68514,-73.95976,Entire home/apt,76,272
2,2,5121,BlissArtsSpace!,40.68688,-73.95596,Private room,60,365
3,3,5136,"Spacious Brooklyn Duplex, Patio + Garden",40.66120,-73.99423,Entire home/apt,175,352
4,4,5178,Large Furnished Room Near B'way,40.76489,-73.98493,Private room,79,346


In [3]:
data_crime = pd.read_csv('sampled crime.csv')
data_crime.head()

,CMPLNT_FR_DT,Latitude,Longitude,PATROL_BORO
0,2019,40.747044,-73.987128,PATROL BORO MAN SOUTH
1,2019,40.690658,-73.864497,PATROL BORO QUEENS SOUTH
2,2019,40.719956,-73.938616,PATROL BORO BKLYN NORTH
3,2019,40.679284,-73.929059,PATROL BORO BKLYN NORTH
4,2019,40.848972,-73.908879,PATROL BORO BRONX


In [4]:
data_airbnb = data_airbnb[['name', 'latitude', 'longitude']].copy()
data_airbnb.head()

,name,latitude,longitude
0,Skylit Midtown Castle,40.75362,-73.98377
1,"Whole flr w/private bdrm, bath & kitchen(pls r...",40.68514,-73.95976
2,BlissArtsSpace!,40.68688,-73.95596
3,"Spacious Brooklyn Duplex, Patio + Garden",40.66120,-73.99423
4,Large Furnished Room Near B'way,40.76489,-73.98493


In [5]:
data_crime = data_crime[['PATROL_BORO', 'Latitude', 'Longitude']].copy()
data_crime.head()

,PATROL_BORO,Latitude,Longitude
0,PATROL BORO MAN SOUTH,40.747044,-73.987128
1,PATROL BORO QUEENS SOUTH,40.690658,-73.864497
2,PATROL BORO BKLYN NORTH,40.719956,-73.938616
3,PATROL BORO BKLYN NORTH,40.679284,-73.929059
4,PATROL BORO BRONX,40.848972,-73.908879


In [6]:
data_airbnb[['lat_radians_airbnb','long_radians_airbnb']] = (
    np.radians(data_airbnb.loc[:,['latitude','longitude']])
)

data_crime[['lat_radians_crime','long_radians_crime']] = (
    np.radians(data_crime.loc[:,['Latitude','Longitude']])
)
data_airbnb.head()
data_crime.head()

,PATROL_BORO,Latitude,Longitude,lat_radians_crime,long_radians_crime
0,PATROL BORO MAN SOUTH,40.747044,-73.987128,0.711170,-1.291319
1,PATROL BORO QUEENS SOUTH,40.690658,-73.864497,0.710186,-1.289179
2,PATROL BORO BKLYN NORTH,40.719956,-73.938616,0.710697,-1.290472
3,PATROL BORO BKLYN NORTH,40.679284,-73.929059,0.709987,-1.290305
4,PATROL BORO BRONX,40.848972,-73.908879,0.712949,-1.289953


In [7]:
dist = sklearn.neighbors.DistanceMetric.get_metric('haversine')
dist_matrix = (dist.pairwise
    (data_airbnb[['lat_radians_airbnb','long_radians_airbnb']],
     data_crime[['lat_radians_crime','long_radians_crime']])*3959
)
# Note that 3959 is the radius of the earth in miles
df_dist_matrix = (
    pd.DataFrame(dist_matrix,index=data_airbnb['name'], 
                 columns=data_crime['PATROL_BORO'])
)

print(df_dist_matrix.head())

PATROL_BORO                                         PATROL BORO MAN SOUTH  \
name                                                                        
Skylit Midtown Castle                                            0.487233   
Whole flr w/private bdrm, bath & kitchen(pls read)               4.511151   
BlissArtsSpace!                                                  4.466145   
Spacious Brooklyn Duplex, Patio + Garden                         5.943235   
Large Furnished Room Near B'way　                                 1.238504   

PATROL_BORO                                         PATROL BORO QUEENS SOUTH  \
name                                                                           
Skylit Midtown Castle                                               7.611837   
Whole flr w/private bdrm, bath & kitchen(pls read)                  5.005821   
BlissArtsSpace!                                                     4.799222   
Spacious Brooklyn Duplex, Patio + Garden                    

In [8]:
result_crime = df_dist_matrix[df_dist_matrix < 0.5].count(axis='columns')

In [9]:
data_station = pd.read_csv('station_geom.csv')
data_station.head()

,NAME,longitude,latitude
0,Astor Pl,-73.991070,40.730054
1,Canal St,-74.000193,40.718803
2,50th St,-73.983849,40.761728
3,Bergen St,-73.974999,40.680862
4,Pennsylvania Ave,-73.894886,40.664714


In [10]:
data_station[['lat_radians_crime','long_radians_crime']] = (
    np.radians(data_station.loc[:,['latitude', 'longitude']])
)

In [11]:
dist = sklearn.neighbors.DistanceMetric.get_metric('haversine')
dist_matrix_station = (dist.pairwise
    (data_airbnb[['lat_radians_airbnb','long_radians_airbnb']],
     data_station[['lat_radians_crime','long_radians_crime']])*3959
)
# Note that 3959 is the radius of the earth in miles
df_dist_matrix_station = (
    pd.DataFrame(dist_matrix_station,index=data_airbnb['name'], 
                 columns=data_station['NAME'])
)

print(df_dist_matrix_station.head())

NAME                                                Astor Pl  Canal St  \
name                                                                     
Skylit Midtown Castle                               1.672600  2.554816   
Whole flr w/private bdrm, bath & kitchen(pls read)  3.510125  3.145867   
BlissArtsSpace!                                     3.504506  3.199112   
Spacious Brooklyn Duplex, Patio + Garden            4.760525  3.992471   
Large Furnished Room Near B'way　                    2.428448  3.283219   

NAME                                                 50th St  Bergen St  \
name                                                                      
Skylit Midtown Castle                               0.560258   5.048334   
Whole flr w/private bdrm, bath & kitchen(pls read)  5.440316   0.851465   
BlissArtsSpace!                                     5.374064   1.080803   
Spacious Brooklyn Duplex, Patio + Garden            6.967489   1.691619   
Large Furnished Room Near B'way

In [12]:
result_station = df_dist_matrix_station[df_dist_matrix_station < 0.3].count(axis='columns')

In [13]:
# result = pd.concat([df1, s1], axis=1)
# test = pd.concat([data_airbnb_copy, result_station], axis = 1)

In [29]:
# data_airbnb_copy.append(result_crime, ignore_index=True)
# data_airbnb_copy['e'] = result_crime
# data_airbnb_copy['crime'] = result_crime

df_result_crime = result_crime.to_frame()
df_result_crime.reset_index(inplace=True)
df_result_crime = df_result_crime.rename(columns = {'index':'name'})
# print(df_result_crime.head())
df_result_station = result_station.to_frame()
df_result_station.reset_index(inplace=True)
df_result_station = df_result_station.rename(columns = {'index':'name'})
# print(df_result_station.head())
# print(df_result_station.shape)
# print(len(df_result_crime.index))
# merged_with_crime = pd.merge(left=data_airbnb_copy, right=df_result_crime, left_on='name', right_on='name')
# merged_with_crime = pd.merge(left=data_airbnb_copy, right=df_result_crime, on='name')
merged_with_crime = pd.merge(data_airbnb_copy, df_result_crime, left_index=True, right_index=True)
# print(merged_with_crime.shape)
# merged_with_station = pd.merge(left=merged_with_crime, right=df_result_station, left_on='name', right_on='name')
merged_with_station = pd.merge(merged_with_crime, df_result_station, left_index=True, right_index=True)
# print(merged_with_station.shape)
# print(merged_with_station.head())
merged_with_station = merged_with_station.drop(['name', 'name_y'], axis=1)
# merged_with_station.drop(columns=['B', 'C'])
merged_with_station = merged_with_station.rename(columns={"name_x": "name", "0_x": "number_of_crime", "0_y": "number_of_station"})
print(merged_with_station.shape)
print(merged_with_station.head())

(25648, 9)
     id                                               name  latitude  \
0  2595                              Skylit Midtown Castle  40.75362   
1  3831  Whole flr w/private bdrm, bath & kitchen(pls r...  40.68514   
2  5121                                    BlissArtsSpace!  40.68688   
3  5136           Spacious Brooklyn Duplex, Patio + Garden  40.66120   
4  5178                   Large Furnished Room Near B'way　  40.76489   

   longitude        room_type  price  availability_365  number_of_crime  \
0  -73.98377  Entire home/apt    175               365              257   
1  -73.95976  Entire home/apt     76               272               48   
2  -73.95596     Private room     60               365               52   
3  -73.99423  Entire home/apt    175               352               21   
4  -73.98493     Private room     79               346              131   

   number_of_station  
0                  7  
1                  1  
2                  2  
3            

In [30]:
merged_with_station.to_csv('Airbnb_with_station_crime.csv')